In [37]:
#  Find K Means of fictional device status locations
from pprint import pprint

In [38]:
def distance(p1,p2):  
    return (p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2


# The sum of two points
def add_points(p1,p2):
    return [(p1[0] + p2[0]), (p1[1] + p2[1])]


# The files with device status data
filename = "/fictional/devicestatus_etl/*"

In [46]:
# K is the number of means (center points of clusters) to find
K = 80


# Split by delimiter, parse device status records into [latitude,longitude]
# Filter out records where lat/long is unavailable -- ie: 0/0 points
rdd = sc.textFile(filename) \
        .map(lambda s: s.split(',')) \
        .filter(lambda values: values[3]!='0' and values[4]!='0') \
        .map(lambda values: (float(values[3]), float(values[4])))

        
rdd.cache()
# rdd.count() #94039
# rdd.take(K)


# start with K randomly selected points from the dataset
sample = rdd.sample(False, .001, 1988).take(K) # Valores aleatórios iniciais
# pprint(list(sample))


def agrupe((px,py), sample):
    d = float("+inf")
    near = (0,0)
    for x,y in sample:
        new_d = distance([px,py],[x,y])
        if new_d <= d:
            near = (x,y)
            d = new_d
    return near,((px,py), 1)


# ConvergeDist -- the threshold "distance" between iterations at which we decide we are done
convergeDist = max(min(1/K, .01), .001)


# loop until the total distance between one iteration's points and the next is less than 
# the convergence distance specified
tempDist = float("+inf")


while tempDist > convergeDist:
    sample = rdd.map(lambda p: agrupe(p,sample)) \
                .reduceByKey(lambda k1,k2: (add_points(k1[0],k2[0]), k1[1]+k2[1])) \
                .mapValues(lambda (p,q): (p[0]/q,p[1]/q)) \
#         .take(K)
    
    tempDist = sum(list(sample.map(lambda (a,b): distance(a,b)).take(K)))
    print('Distancia Média: ', tempDist)
    
    sample = sample.values().take(K)

Distancia Média:  20.0008451384
Distancia Média:  8.13057155033
Distancia Média:  0.419110188831
Distancia Média:  0.222855924601
Distancia Média:  0.0474823392142
Distancia Média:  0.0525792925589
Distancia Média:  0.0121069723554
Distancia Média:  0.00459948494552
Distancia Média:  0.00405326202804
Distancia Média:  0.00454817964828
Distancia Média:  0.00745126601867
Distancia Média:  0.0150336284688
Distancia Média:  0.0408243586057
Distancia Média:  0.0590179521311
Distancia Média:  0.00688949768
Distancia Média:  0.00174892884683
Distancia Média:  0.00213901835366
Distancia Média:  0.00315192767543
Distancia Média:  0.00190277497108
Distancia Média:  0.00196079036863
Distancia Média:  0.000752525242963


In [48]:
# Display the final points
pprint(sample)

[(37.24597671824258, -121.9488687077137),
 (34.44989999670529, -117.3429929596028),
 (36.20959158274142, -119.00160645757133),
 (39.711147212056176, -119.49929652925859),
 (40.13196687167813, -122.32649781023237),
 (45.83817924049371, -121.84970909271166),
 (33.59824274237049, -111.7456586560545),
 (36.58480105564365, -120.87718973793957),
 (34.31466290366444, -118.51133701249793),
 (39.56049101444568, -120.92795485719817),
 (33.87580140337335, -117.47836502702526),
 (36.22911684400944, -113.8282149106722),
 (33.90295245799995, -108.81985111778866),
 (32.22388852885904, -111.63668694159439),
 (37.531446718548885, -121.91605104496782),
 (36.30039104648227, -114.77114012959953),
 (38.32931211286598, -122.25812827760988),
 (45.28434654289552, -117.68504186273312),
 (37.67475519977155, -121.43624274273088),
 (37.88338079566217, -121.88567258119325),
 (38.02864616883508, -121.58765445866764),
 (38.62701083397112, -122.17107558840803),
 (38.58416974077095, -122.7368253081486),
 (34.034854567

   .